# Day 20 In-class Assignment: Fitting Exoplanent Data with MCMC

<img src="https://upload.wikimedia.org/wikipedia/commons/5/59/Orbit3.gif" width=300px>

## Problem Description

In the assignment repository for Day 21, along with this notebook, you should see a file called '51_Pegasi_RadVel.dat'. This file contains radial velocity data (which is the line-of-sight velocity) for a star called "51 Pegasi", a star located in the Pegasus constellation. This radial velocity data indicates that the star is "wobbling" towards and away from us as we look at it. This stellar wobble is actually a sign that there is another relatively massive object orbiting the star and gravititational pulling it around (see the image above). In fact, this star happens to be home to the first extrasolar planet ever observed around a main-sequence star (a star that currently fusing hydrogen into helium in it's core much like our Sun). This planet was originally given the very original name of "[51 Pegasi b](https://en.wikipedia.org/wiki/51_Pegasi_b)".

Astronomers were able to observe that there is a planet orbiting 51 Pegasi through a method referred to as [Doppler Spectroscopy](https://en.wikipedia.org/wiki/Doppler_spectroscopy). Basically, if you can measure the radial velocity "wobbles" of the star and you understand a bit about orbital mechanics, you can fit a model to the data and determine the properties of the object that is causing the wobbles. You're going to do this in-class today. And you're going to do it using the MCMC method we learned in the last class period!

(Side note: it turns out that one of the previous CMSE 202 instructors has a bit of experience with [modeling the orbits of stars and using MCMC methods to estimate parameters](http://iopscience.iop.org/article/10.1086/588545/meta). Clearly this topic was chosen by coincidence)

### Agenda for today's class

</p>

1. Loading the data
1. Plotting the data
1. Defining the model that fits the data
1. Fitting the data with the model (MCMC)
1. Visualizing the trace
1. Find the best fit values
1. Comparing the best fit model to the data
1. Understanding the physical meaning of our best fit parameters
---

## 1. Loading the data

First things first, we need to load up the data so that we can compute a model and fit the data. Using whatever means necessary, load the data into your notebook in the cell below. The easiest option might be to use `np.loadtxt()`. Inspect the file to make sure you understand what each column is.

In [ ]:
# Put your code here


## 2. Plotting the data

Now that you have the data loaded in, **make a plot of the radial velocities as a function of time**. The observations of 51 Pegasi are not at regular intervals, so you'll probably want to make this a scatter plot. 

*Note*: This dataset defines time in terms of "[Julian Days](https://en.wikipedia.org/wiki/Julian_day)", which is a weird time scale that astronomers use. The main thing you need to know is that 1 Julian Day is equivalent to 1 normal day. **You should redefine the time array of Julian Days by subtracting off the Julian Date of the first observation so that our first data point starts at t=0.**

In [ ]:
# Put your plot here


**Huh.** Well, that probably's isn't a super useful plot, is it? If we wanted to know if there was any sort of meaningful pattern in the data, this plot doesn't really provide us with much insight.

We're going to try to make a much more useful plot, but we have to cheat a little. Let's assume that we can make an educated guess about the orbital period of the hidden exoplanet, and assume it to be $P_{\mathrm{guess}} = 4.23$ days (This is a pretty good guess... there's actually a method for determining the period from just the data alone, but we won't get into that here). Using this information, we can "fold" the data points together based on this orbital period, by redefined our "time" in the following way:

$$ t_{\mathrm{phase}} = \frac{t \bmod P}{P} $$

where $t$ is our original time array and $\bmod$ is the [modulus operator](https://en.wikipedia.org/wiki/Modulo_operation). This will show us how the radial velocities change over just one orbital period. **Try redefining your time values in this way and plot the radial velocities as a function of orbital phase. Add the error bars from the dataset to this plot as well** You should find that you get something that looks like this:

<img src="https://i.imgur.com/jMCPu94.png" width=400px>

**Clearly this indicates that a pattern is embedded the data!**

In [ ]:
# Put your code here


## 3. Defining the model that fits the data

Now that we know that there is clearly as pattern in the data, it's likely that there is a model that explains the observed features. We can actually define the value of the radial velocity in the following way:

$$ v_{\mathrm{radial}} = A \sin(B~t_{\mathrm{phase}}+C) $$

where $t_{\mathrm{phase}}$ is the "folded" time we created above and $A$, $B$, and $C$ are the unknown parameters of our model.

**Write a function that takes as inputs the folded time array, A, B, and C. The function should return the predicted values for $v_{\mathrm{radial}}$.**

In [ ]:
# Put your function here


## 4. Fitting the data with the model

Now we're going to try to use our Markov Chain Monte Carlo (MCMC) method to find the best fit parameters for our model given the data we have for 51 Pegasi.

For reference, here is the code from the previous assignment that computes the $\chi^2$ error between data and a model:

In [ ]:
def calculate_error(ys_actual, ys_model, sigma):
    """
    Calculate the chi-squared error between two sets of data
    """
    return ((ys_actual-ys_model)**2/(2*sigma**2)).sum()/(ys_actual.size)

And, as another reminder, here is the general approach for how the Metropolis-Hastings MCMC method works, but keep in mind that you need to implement this for your radial velocity model, not for the parabola we looked at from last time:

>The general idea behind MCMC fitting of our data is that we will start from a guess at our model parameters and "walk" in random directions in parameter space in a way that **on average gets us closer to the best fit to the data.**  We keep track of the points that we've sampled over time (we call this the "trace" of the data), and use those to create a distribution.  The distribution shows us how likely each set of model parameters is to fit the data.

>We're going to think about how this works using the model from our pre-class assignment as an example.
Specifically, you're going to revisit fitting data of the form
$$
    f(x) = W x^2 + I
$$
by intelligently searching for optimal $W$ and $I$ (_width_ and _intercept_ of the parabola) values. Specifically, Metropolis-Hastings fitting consists of these steps:
1. Start with an initial guess of the model parameters, $(W_0, I_0)$.
2. Take a (potential) "step" from $(W_0, I_0)$ in a random direction to produce $(W_1, I_1)$
3. Calculate the the ["reduced chi-squared"](https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic) values (as done in the pre-class), $\chi_0^2$ and $\chi_1^2$, for the parabolas defined by $(W_0, I_0)$ and $(W_1, I_1)$.
4. Calculate an acceptance probability, $P = e^{-\chi_1^2}/e^{-\chi_0^2}$, as the ratio of two *likelihood functions* (the exponentials).
5. Uniformly generate a random number $r \in [0, 1)$. If $r < P$, "accept" $(W_1, I_1)$ as the next initial guess. Otherwise, discard $(W_1, I_1)$ and generate a new potential step from $(W_0, I_0)$.
6. Repeat this process until you've generated as many points as you care to, keeping in mind that you need to have _enough_ points for the walker to find the right values.


Note that for this problem, using the same stepsize for every parameter probably doesn't make sense since the values themselves differ substantially. Instead, **use a step size that is no larger than 10% of the current parameter value**.

You need to make sure you feed your $t_{phase}$ values into your model and not the standard values that are in days!

In [ ]:
# Here's so code to get you started:

# Total number of points we're going to sample
num_sample_points = 100000

# Weighting factor for the random step
step_size = 0.1

# Provide an initial guess for the width and intercept parameters
# As we move the walker around, we'll use these same lists to 
# store our new values so that we can visualize the path later!
# (Note: that means you'll want to append new values to these lists!)
As = [60.0] # We can eyeball this one from the plot
Bs = [2.5*np.pi] # We'll choose something that will allow the walker to wander for a bit
Cs = [-0.5] # this is a guess!

# Put the rest of your code here!


## 5. Visualizing the "trace"

Now we want to see where our walker went! **Make a plot for each of the following**:

* all of your B values versus your A values
* all of your C values versus your B values
* all of your C values versus your A values

**Add points or text to show where your walker started and where it stopped**.

In [ ]:
# Put your answer here


## 6. Finding the best fit values

As we saw in the last class, just displaying the path of the walker isn't a great way of visualizing the data. You should try to make the same sorts of 2D histogram plots you made last time that show where the walker spent most of it's time.

**You can also use the 2D histogram plots to find where the best fit parameters are**.

You should find that you get something that looks like this:

<img src="https://i.imgur.com/ktlniN7.png" width=600px>

**You need to make at least 2 of these plots to find the best fit values for all three parameters!**

## 7. Comparing the best fit model to the data

Now that you have all of the best fit values for $A$, $B$, and $C$. **Plot the data and the error bars and then overplot the best fit model for an evenly spaced array of $t_{\mathrm{phase}}$ values spanning 0 to 1**.

In [ ]:
# Put your code here


## 8. Understanding the physical meaning of our best fit parameters

Of course, these random parameters, $A$, $B$, and $C$ are not particularly meaningful, so **what do these parameters mean physically?**

These parameters are related to the physical system in the following ways:

* $A$ is just the semi-amplitude of the radial velocity curve, which is normally referred to as $K$.
* $B$ is the frequency of the sine wave, which we can use to the compute the period, $P_{\mathrm{days}} = \frac{2\pi}{B}P_{\mathrm{guess}}$ (where we're rescaling the result by the original period we used to fold the data)
* $-C/B$ is the phase shift for our sine curve, which we don't need for any of the physical values we care about

If we then know the mass of the host star 51 Pegasi, $M_{\star}$, then we have enough information to estimate the  the mass of the planet, $M_{\mathrm{p}}$ and the distance the planet is away from the star, $r$ (if we assume a circular orbit).

These are given by the following set of equations:

$$ r = \big[\frac{G~M_{\star}~P^2}{4~\pi^2}\big]^{1/3} $$
$$ v_{\mathrm{planet}} = \sqrt{\frac{G~M_{\star}}{r}} $$
$$ M_{\mathrm{planet}} = \frac{M_{\star}v_{\star}}{v_{\mathrm{planet}}} $$

where if the orbit of the planet is "edge-on", we can assume that $v_{\star} = K$. We also know that the mass of 51 Pegasi is $M_{\mathrm{\star}} = 1.06 M_{\mathrm{sun}}$. **Important note**: in the equation for $r$, the value of that $P$ needs to be in *seconds*.

**Write a function that takes in the values of $A$ and $B$ and *returns* the values for $M_{\mathrm{planet}}$ and $r$.**

In [ ]:
# Put your function here


### Estimating the properties of the planetary system

Now that you have your best fit model and an understanding of the physical meaning of the parameters, the final step is computing the orbital radius and mass of the planet. **Using the function you wrote above**, print the radius of the planet's orbit and the mass of the planet. You might want to convert the mass to Jupiter masses and the radius to astronomical units.

In [ ]:
# Put your code here


**Question**: How do your results compare to the known values for [51 Pegasi b](http://exoplanet.eu/catalog/51_peg_b/)?

*Put your answer here*

## Exploring how changes to the method or the model changes the results

If you have time leftover, explore how changing the initial conditions, the step size factor, or the number of iterations changes your answer. How robust are you results?

Also, notice that the parameters $B$ and $C$ are a clearly a bit coupled -- if I increase $B$ and decrease $C$, or vice versa, I can move along a region of space with a similar "goodness of fit". This actually has to do with the fact that the phase shift is the ratio of $-C/B$, so the result isn't necessarily surprising!

*Record any thoughts you might have here*

----
# Assignment Wrap-up

Fill out the following Google Form before submitting your assignment to D2L!

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/forms/noAwoJd9pEj7A1su1" 
	width="800px" 
	height="600px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

-----
### Congratulations, we're done!

Now, you just need to submit this assignment by uploading it to the course <a href="https://d2l.msu.edu/">Desire2Learn</a> web page for today's submission folder (Don't forget to add your names in the first cell).


&#169; Copyright 2018,  Michigan State University Board of Trustees